In [1]:
#!pip install cbpro

In [2]:
import cbpro
import time
from datetime import datetime
import pandas as pd
from websocket import create_connection
import json
import websocket

# Python client of coinbase pro api exploration

In [3]:
public_client = cbpro.PublicClient()

## Get products 
Get a list of available currency pairs for trading

In [4]:
products=public_client.get_products()
pd.DataFrame(products).head(1)

,id,base_currency,quote_currency,base_min_size,base_max_size,quote_increment,base_increment,display_name,min_market_funds,max_market_funds,margin_enabled,post_only,limit_only,cancel_only,trading_disabled,status,status_message
0,ZRX-USD,ZRX,USD,1.00000000,600000.00000000,0.00000100,0.00001000,ZRX/USD,10,100000,False,False,False,False,False,online,


## Get product order book
Get a list of open orders for a product. The amount of detail shown can be customized with the level parameter.

In [105]:
# Get the order book at the default level.
public_client.get_product_order_book('BTC-USD')
# Get the order book at a specific level.
public_client.get_product_order_book('BTC-USD', level=1)

{'bids': [['48793.38', '0.00006239', 1]],
 'asks': [['48793.39', '0.58199754', 8]],
 'sequence': 22146197501}

## Get product ticker
Snapshot information about the last trade (tick), best bid/ask and 24h volume.

In [6]:
# Get the product ticker for a specific product.
public_client.get_product_ticker(product_id='ETH-USD')

{'trade_id': 89279941,
 'price': '1445.16',
 'size': '0.2',
 'time': '2021-02-28T02:23:56.252278Z',
 'bid': '1445.02',
 'ask': '1445.13',
 'volume': '213605.1349304'}

## Get product trades
List the latest trades for a product.

In [7]:
# Get the product trades for a specific product.
# Returns a generator
public_client.get_product_trades(product_id='ETH-USD')

<generator object PublicClient._send_paginated_message at 0x7f80dd6436d0>

# Get product historic rates 
Historic rates for a product. Rates are returned in grouped buckets based on requested granularity.



In [8]:
public_client.get_product_historic_rates('ETH-USD')
# To include other parameters, see function docstring:
public_client.get_product_historic_rates('ETH-USD', granularity=3600)
#[ time, low, high, open, close, volume ]

[[1614477600, 1448.21, 1466.28, 1464.56, 1450.22, 1843.58334639],
 [1614474000, 1451.42, 1469.96, 1462.15, 1464.57, 6011.82112733],
 [1614470400, 1442.77, 1469.69, 1460.49, 1462.43, 11648.71676646],
 [1614466800, 1420, 1471.33, 1465.4, 1460.57, 41064.90768626],
 [1614463200, 1463, 1511.3, 1507.72, 1464.9, 13296.62830643],
 [1614459600, 1498.07, 1524.3, 1509.9, 1507.73, 11554.49655021],
 [1614456000, 1502.05, 1522.22, 1513.48, 1509.85, 9653.89632113],
 [1614452400, 1490, 1516.41, 1492.34, 1513.49, 14474.71350656],
 [1614448800, 1488.62, 1496.99, 1488.75, 1492.35, 7244.14751053],
 [1614445200, 1469.79, 1489.71, 1471.51, 1488.6, 7870.24875812],
 [1614441600, 1447.08, 1476.51, 1454.24, 1471.44, 8188.13780137],
 [1614438000, 1446.1, 1490, 1473.79, 1454.24, 12966.9237907],
 [1614434400, 1461.46, 1490.37, 1484.2, 1473.79, 5735.14182478],
 [1614430800, 1477.26, 1495, 1487.25, 1484.05, 5432.42152019],
 [1614427200, 1455.01, 1494.84, 1470.23, 1486.82, 5840.41700346],
 [1614423600, 1462.06, 1485.

# Websocket 

## Heartbeat channel
To receive heartbeat messages for specific products once a second subscribe to the heartbeat channel. Heartbeats also include sequence numbers and last trade ids that can be used to verify no messages were missed.

In [93]:
# Parameters are optional
wsClient = cbpro.WebsocketClient(url="wss://ws-feed.pro.coinbase.com",
                                products="BTC-USD",
                                channels=["heartbeat"])
# Do other stuff...

In [94]:
wsClient.start()
time.sleep(10)
wsClient.close()

-- Subscribed! --

{'type': 'subscriptions', 'channels': [{'name': 'heartbeat', 'product_ids': ['BTC-USD']}]}
{'type': 'heartbeat', 'last_trade_id': 0, 'product_id': 'BTC-USD', 'sequence': 22091201388, 'time': '2021-02-28T17:26:38.931664Z'}
{'type': 'heartbeat', 'last_trade_id': 0, 'product_id': 'BTC-USD', 'sequence': 22091201761, 'time': '2021-02-28T17:26:39.931710Z'}
{'type': 'heartbeat', 'last_trade_id': 0, 'product_id': 'BTC-USD', 'sequence': 22091202274, 'time': '2021-02-28T17:26:40.931704Z'}
{'type': 'heartbeat', 'last_trade_id': 0, 'product_id': 'BTC-USD', 'sequence': 22091203264, 'time': '2021-02-28T17:26:41.931663Z'}
{'type': 'heartbeat', 'last_trade_id': 0, 'product_id': 'BTC-USD', 'sequence': 22091204101, 'time': '2021-02-28T17:26:42.931872Z'}
{'type': 'heartbeat', 'last_trade_id': 0, 'product_id': 'BTC-USD', 'sequence': 22091204630, 'time': '2021-02-28T17:26:43.931686Z'}
{'type': 'heartbeat', 'last_trade_id': 0, 'product_id': 'BTC-USD', 'sequence': 22091205190, 'time': '202

## Ticker channel
The ticker channel provides real-time price updates every time a match happens. It batches updates in case of cascading matches, greatly reducing bandwidth requirements.

In [9]:
# Parameters are optional
wsClient = cbpro.WebsocketClient(url="wss://ws-feed.pro.coinbase.com",
                                products="BTC-USD",
                                channels=["ticker"])
# Do other stuff...

In [88]:
wsClient.start()
time.sleep(2)
wsClient.close()

Let's count the messages!
{'type': 'subscriptions', 'channels': [{'name': 'level2', 'product_ids': ['BTC-USD']}]}
{'type': 'snapshot', 'product_id': 'BTC-USD', 'asks': [['43795.92', '1.00678912'], ['43797.14', '0.00107974'], ['43798.11', '0.04396442'], ['43800.24', '0.68300000'], ['43800.29', '0.81696228'], ['43801.01', '0.45681497'], ['43804.35', '0.21179557'], ['43804.37', '0.45683680'], ['43804.38', '0.00119225'], ['43807.18', '0.06051603'], ['43807.19', '0.05817272'], ['43807.90', '0.10000000'], ['43808.76', '0.00120231'], ['43810.64', '0.05000000'], ['43810.72', '0.68300000'], ['43810.78', '0.01496392'], ['43811.49', '0.06000000'], ['43813.14', '0.00121608'], ['43814.15', '0.45681497'], ['43814.55', '0.07462254'], ['43815.36', '0.19144928'], ['43815.69', '0.74000000'], ['43816.34', '0.40800000'], ['43816.52', '0.24260000'], ['43816.59', '0.22830000'], ['43817.52', '0.00121194'], ['43817.99', '0.32910000'], ['43818.89', '0.45655452'], ['43819.78', '1.78545855'], ['43819.79', '0.456

## Level 2 channel 

In [95]:
wsClient = cbpro.WebsocketClient(url="wss://ws-feed.pro.coinbase.com",
                                products=["BTC-USD"],
                                channels=["level2"])

In [96]:
wsClient.start()

-- Subscribed! --

{'type': 'subscriptions', 'channels': [{'name': 'level2', 'product_ids': ['BTC-USD']}]}


In [97]:
wsClient.close()

{'type': 'snapshot', 'product_id': 'BTC-USD', 'asks': [['43675.29', '0.06326868'], ['43685.84', '0.22000000'], ['43686.18', '0.01606000'], ['43686.34', '0.80000000'], ['43687.57', '0.45801781'], ['43693.45', '0.27696153'], ['43693.46', '0.68600000'], ['43693.47', '0.68600000'], ['43693.51', '0.29852032'], ['43693.52', '1.65980908'], ['43693.57', '0.80000000'], ['43693.58', '0.80000000'], ['43693.59', '0.45797005'], ['43694.40', '0.45700000'], ['43695.33', '0.10000000'], ['43698.05', '0.68000000'], ['43698.60', '0.80000000'], ['43698.61', '0.71283855'], ['43698.83', '0.66832922'], ['43699.33', '0.45789740'], ['43701.05', '0.80000000'], ['43701.13', '0.06000000'], ['43703.09', '0.20000000'], ['43704.02', '0.45771542'], ['43706.67', '0.70000000'], ['43706.69', '0.02100000'], ['43707.62', '0.02100000'], ['43708.43', '0.02100000'], ['43708.60', '0.45804176'], ['43709.20', '1.00000000'], ['43710.24', '0.82000000'], ['43714.11', '1.20000000'], ['43715.25', '0.04120000'], ['43716.44', '0.23000

## Matches channel 

In [99]:
wsClient = cbpro.WebsocketClient(url="wss://ws-feed.pro.coinbase.com",
                                products=["BTC-USD"],
                                channels=["matches"])

In [100]:
wsClient.start()
time.sleep(2)
wsClient.close()

-- Subscribed! --

{'type': 'subscriptions', 'channels': [{'name': 'matches', 'product_ids': ['BTC-USD']}]}
{'type': 'last_match', 'trade_id': 139554299, 'maker_order_id': '08e91343-bcfd-41b9-9bf7-e11fa13dd488', 'taker_order_id': '4ab946f7-dd83-41ea-9e7e-a428ef292ea9', 'side': 'buy', 'size': '0.02420103', 'price': '43641.81', 'product_id': 'BTC-USD', 'sequence': 22091296651, 'time': '2021-02-28T17:29:06.724875Z'}
{'type': 'match', 'trade_id': 139554300, 'maker_order_id': '4b5c279c-96a0-4d4a-b67a-eee1a58cb2d6', 'taker_order_id': '2fa19bc6-b696-469a-b0d3-8a6e044d866d', 'side': 'sell', 'size': '0.00221168', 'price': '43641.82', 'product_id': 'BTC-USD', 'sequence': 22091296745, 'time': '2021-02-28T17:29:06.993241Z'}
{'type': 'match', 'trade_id': 139554301, 'maker_order_id': '8b785b73-bd05-46f5-a6d9-8197646994e9', 'taker_order_id': 'b13c9d50-1b45-4160-91e4-8538dd542a55', 'side': 'sell', 'size': '0.001094', 'price': '43641.82', 'product_id': 'BTC-USD', 'sequence': 22091297000, 'time': '2021-

In [104]:
class MyWebsocketClient(cbpro.WebsocketClient):
    def on_open(self):
        self.url = "wss://ws-feed.pro.coinbase.com/"
        self.products = ["BTC-USD"]
        self.message_count = 0
        self.channels=['level2']
        self.all=[]

    def on_message(self, msg):
        print(msg)
        self.all+=[msg]
        self.message_count += 1

    def on_close(self):
        print("-- Goodbye! --")


wsClient = MyWebsocketClient()
wsClient.start()
time.sleep(5)
wsClient.close()

{'type': 'subscriptions', 'channels': [{'name': 'level2', 'product_ids': ['BTC-USD']}]}
{'type': 'snapshot', 'product_id': 'BTC-USD', 'asks': [['48782.57', '1.12099497'], ['48784.94', '0.02528644'], ['48785.10', '0.32000000'], ['48785.47', '0.00118088'], ['48785.73', '0.01812213'], ['48788.50', '0.61400000'], ['48789.15', '0.00110975'], ['48791.88', '0.61400000'], ['48792.27', '0.00531980'], ['48794.00', '0.10000000'], ['48794.01', '0.00200000'], ['48794.13', '0.04824387'], ['48794.88', '0.00245160'], ['48795.31', '0.00119434'], ['48797.06', '0.00119434'], ['48799.98', '0.10000000'], ['48800.00', '0.18898941'], ['48800.37', '0.06000000'], ['48801.21', '0.00710219'], ['48801.89', '0.30744359'], ['48804.48', '0.00200000'], ['48806.07', '0.00119136'], ['48806.60', '0.23530000'], ['48807.23', '0.40992479'], ['48809.32', '0.08275643'], ['48810.01', '0.42925684'], ['48810.98', '0.41008037'], ['48812.25', '0.01900000'], ['48813.12', '0.01900000'], ['48814.55', '0.01900000'], ['48814.92', '0.4

{'type': 'l2update', 'product_id': 'BTC-USD', 'changes': [['buy', '48783.00', '0.00824435']], 'time': '2021-03-01T18:02:53.247730Z'}
{'type': 'l2update', 'product_id': 'BTC-USD', 'changes': [['sell', '48791.96', '0.00000000']], 'time': '2021-03-01T18:02:53.252670Z'}
{'type': 'l2update', 'product_id': 'BTC-USD', 'changes': [['buy', '48787.95', '0.00000000']], 'time': '2021-03-01T18:02:53.252689Z'}
{'type': 'l2update', 'product_id': 'BTC-USD', 'changes': [['buy', '48781.01', '0.00000000']], 'time': '2021-03-01T18:02:53.258871Z'}
{'type': 'l2update', 'product_id': 'BTC-USD', 'changes': [['sell', '48796.38', '0.00894705']], 'time': '2021-03-01T18:02:53.264358Z'}
{'type': 'l2update', 'product_id': 'BTC-USD', 'changes': [['buy', '48787.67', '0.32000000']], 'time': '2021-03-01T18:02:53.264413Z'}
{'type': 'l2update', 'product_id': 'BTC-USD', 'changes': [['sell', '48796.11', '0.32000000']], 'time': '2021-03-01T18:02:53.264455Z'}
{'type': 'l2update', 'product_id': 'BTC-USD', 'changes': [['buy', 

{'type': 'l2update', 'product_id': 'BTC-USD', 'changes': [['sell', '48792.85', '0.00000000']], 'time': '2021-03-01T18:02:53.493948Z'}
{'type': 'l2update', 'product_id': 'BTC-USD', 'changes': [['buy', '48658.76', '0.00000000']], 'time': '2021-03-01T18:02:53.494162Z'}
{'type': 'l2update', 'product_id': 'BTC-USD', 'changes': [['sell', '48905.10', '0.05960000']], 'time': '2021-03-01T18:02:53.494350Z'}
{'type': 'l2update', 'product_id': 'BTC-USD', 'changes': [['buy', '48786.19', '0.06424000']], 'time': '2021-03-01T18:02:53.495248Z'}
{'type': 'l2update', 'product_id': 'BTC-USD', 'changes': [['sell', '48794.32', '0.00000000']], 'time': '2021-03-01T18:02:53.495970Z'}
{'type': 'l2update', 'product_id': 'BTC-USD', 'changes': [['sell', '48794.21', '0.32000000']], 'time': '2021-03-01T18:02:53.498683Z'}
{'type': 'l2update', 'product_id': 'BTC-USD', 'changes': [['buy', '48662.23', '0.00000000']], 'time': '2021-03-01T18:02:53.504510Z'}
{'type': 'l2update', 'product_id': 'BTC-USD', 'changes': [['buy',

{'type': 'l2update', 'product_id': 'BTC-USD', 'changes': [['buy', '48785.83', '0.02424000']], 'time': '2021-03-01T18:02:53.693017Z'}
{'type': 'l2update', 'product_id': 'BTC-USD', 'changes': [['buy', '48785.62', '0.00825038']], 'time': '2021-03-01T18:02:53.693155Z'}
{'type': 'l2update', 'product_id': 'BTC-USD', 'changes': [['buy', '48739.60', '0.00000000']], 'time': '2021-03-01T18:02:53.694792Z'}
{'type': 'l2update', 'product_id': 'BTC-USD', 'changes': [['buy', '48786.31', '0.00000000']], 'time': '2021-03-01T18:02:53.696407Z'}
{'type': 'l2update', 'product_id': 'BTC-USD', 'changes': [['buy', '48785.94', '0.06424000']], 'time': '2021-03-01T18:02:53.697233Z'}
{'type': 'l2update', 'product_id': 'BTC-USD', 'changes': [['sell', '48793.85', '0.00000000']], 'time': '2021-03-01T18:02:53.697428Z'}
{'type': 'l2update', 'product_id': 'BTC-USD', 'changes': [['sell', '48920.66', '0.00000000']], 'time': '2021-03-01T18:02:53.698413Z'}
{'type': 'l2update', 'product_id': 'BTC-USD', 'changes': [['buy', '

{'type': 'l2update', 'product_id': 'BTC-USD', 'changes': [['sell', '48792.35', '0.00000000']], 'time': '2021-03-01T18:02:53.899327Z'}
{'type': 'l2update', 'product_id': 'BTC-USD', 'changes': [['sell', '48792.72', '0.00000000']], 'time': '2021-03-01T18:02:53.901507Z'}
{'type': 'l2update', 'product_id': 'BTC-USD', 'changes': [['sell', '48794.41', '0.00894705']], 'time': '2021-03-01T18:02:53.902144Z'}
{'type': 'l2update', 'product_id': 'BTC-USD', 'changes': [['sell', '48819.76', '0.00000000']], 'time': '2021-03-01T18:02:53.902660Z'}
{'type': 'l2update', 'product_id': 'BTC-USD', 'changes': [['buy', '48671.85', '0.00000000']], 'time': '2021-03-01T18:02:53.905710Z'}
{'type': 'l2update', 'product_id': 'BTC-USD', 'changes': [['buy', '48785.77', '0.06592000']], 'time': '2021-03-01T18:02:53.907213Z'}
{'type': 'l2update', 'product_id': 'BTC-USD', 'changes': [['sell', '48794.41', '0.01789410']], 'time': '2021-03-01T18:02:53.910965Z'}
{'type': 'l2update', 'product_id': 'BTC-USD', 'changes': [['sell

...

{'type': 'l2update', 'product_id': 'BTC-USD', 'changes': [['sell', '48883.53', '0.01658799']], 'time': '2021-03-01T18:02:55.526570Z'}
{'type': 'l2update', 'product_id': 'BTC-USD', 'changes': [['buy', '48782.45', '0.00000000']], 'time': '2021-03-01T18:02:55.526948Z'}
{'type': 'l2update', 'product_id': 'BTC-USD', 'changes': [['sell', '48786.96', '4.82258219']], 'time': '2021-03-01T18:02:55.530297Z'}
{'type': 'l2update', 'product_id': 'BTC-USD', 'changes': [['sell', '48786.95', '1.46000000']], 'time': '2021-03-01T18:02:55.531989Z'}
{'type': 'l2update', 'product_id': 'BTC-USD', 'changes': [['sell', '48914.60', '0.01000000']], 'time': '2021-03-01T18:02:55.532479Z'}
{'type': 'l2update', 'product_id': 'BTC-USD', 'changes': [['sell', '48786.87', '0.06592000']], 'time': '2021-03-01T18:02:55.535582Z'}
{'type': 'l2update', 'product_id': 'BTC-USD', 'changes': [['sell', '48786.59', '0.32000000']], 'time': '2021-03-01T18:02:55.535593Z'}
{'type': 'l2update', 'product_id': 'BTC-USD', 'changes': [['buy

{'type': 'l2update', 'product_id': 'BTC-USD', 'changes': [['buy', '48754.78', '0.09704702']], 'time': '2021-03-01T18:02:55.728593Z'}
{'type': 'l2update', 'product_id': 'BTC-USD', 'changes': [['sell', '48845.67', '0.00000000']], 'time': '2021-03-01T18:02:55.731573Z'}
{'type': 'l2update', 'product_id': 'BTC-USD', 'changes': [['sell', '48778.41', '0.00000000']], 'time': '2021-03-01T18:02:55.736534Z'}
{'type': 'l2update', 'product_id': 'BTC-USD', 'changes': [['sell', '48894.49', '1.55000000']], 'time': '2021-03-01T18:02:55.747655Z'}
{'type': 'l2update', 'product_id': 'BTC-USD', 'changes': [['buy', '48744.69', '0.62000000']], 'time': '2021-03-01T18:02:55.747655Z'}
{'type': 'l2update', 'product_id': 'BTC-USD', 'changes': [['sell', '48818.75', '0.62000000']], 'time': '2021-03-01T18:02:55.747655Z'}
{'type': 'l2update', 'product_id': 'BTC-USD', 'changes': [['buy', '48668.95', '1.55000000']], 'time': '2021-03-01T18:02:55.747655Z'}
{'type': 'l2update', 'product_id': 'BTC-USD', 'changes': [['sell'

{'type': 'l2update', 'product_id': 'BTC-USD', 'changes': [['sell', '48815.58', '0.00000000']], 'time': '2021-03-01T18:02:55.929455Z'}
{'type': 'l2update', 'product_id': 'BTC-USD', 'changes': [['sell', '48781.82', '0.00000000']], 'time': '2021-03-01T18:02:55.930217Z'}
{'type': 'l2update', 'product_id': 'BTC-USD', 'changes': [['sell', '48774.40', '0.01439885']], 'time': '2021-03-01T18:02:55.935322Z'}
{'type': 'l2update', 'product_id': 'BTC-USD', 'changes': [['sell', '48777.52', '0.32000000']], 'time': '2021-03-01T18:02:55.935695Z'}
{'type': 'l2update', 'product_id': 'BTC-USD', 'changes': [['sell', '48777.80', '0.06592000']], 'time': '2021-03-01T18:02:55.938957Z'}
{'type': 'l2update', 'product_id': 'BTC-USD', 'changes': [['buy', '48723.50', '0.00900820']], 'time': '2021-03-01T18:02:55.940618Z'}
{'type': 'l2update', 'product_id': 'BTC-USD', 'changes': [['sell', '48777.87', '0.50000000']], 'time': '2021-03-01T18:02:55.942624Z'}
{'type': 'l2update', 'product_id': 'BTC-USD', 'changes': [['buy

{'type': 'l2update', 'product_id': 'BTC-USD', 'changes': [['sell', '48852.37', '0.00000000']], 'time': '2021-03-01T18:02:56.127787Z'}
{'type': 'l2update', 'product_id': 'BTC-USD', 'changes': [['sell', '48912.61', '0.01000000']], 'time': '2021-03-01T18:02:56.127828Z'}
{'type': 'l2update', 'product_id': 'BTC-USD', 'changes': [['buy', '48774.53', '0.06592000']], 'time': '2021-03-01T18:02:56.128383Z'}
{'type': 'l2update', 'product_id': 'BTC-USD', 'changes': [['sell', '48786.14', '0.32000000']], 'time': '2021-03-01T18:02:56.128489Z'}
{'type': 'l2update', 'product_id': 'BTC-USD', 'changes': [['sell', '48820.71', '0.00000000']], 'time': '2021-03-01T18:02:56.128940Z'}
{'type': 'l2update', 'product_id': 'BTC-USD', 'changes': [['sell', '48800.37', '0.00000000']], 'time': '2021-03-01T18:02:56.129401Z'}
{'type': 'l2update', 'product_id': 'BTC-USD', 'changes': [['buy', '48681.06', '0.00000000']], 'time': '2021-03-01T18:02:56.129740Z'}
{'type': 'l2update', 'product_id': 'BTC-USD', 'changes': [['sell

Connection is already closed. - data: None
-- Goodbye! --


In [74]:
wsClient.all[2]

{'type': 'l2update',
 'product_id': 'BTC-USD',
 'changes': [['buy', '45857.77', '0.22400000']],
 'time': '2021-02-28T02:36:53.233644Z'}

In [87]:
wsClient.all[1]['bids']

[['45907.18', '0.06700000'],
 ['45907.17', '0.05000000'],
 ['45905.17', '0.00118825'],
 ['45900.42', '0.02853227'],
 ['45900.41', '0.01606000'],
 ['45900.08', '0.01648000'],
 ['45900.02', '0.10000000'],
 ['45900.01', '0.10000000'],
 ['45900.00', '0.03495731'],
 ['45899.70', '0.02333669'],
 ['45896.16', '0.04000000'],
 ['45896.12', '0.11239774'],
 ['45896.11', '0.01404141'],
 ['45895.78', '0.05791840'],
 ['45894.66', '0.06000000'],
 ['45892.50', '0.23586018'],
 ['45892.49', '0.06500000'],
 ['45892.47', '0.00332526'],
 ['45892.27', '0.04108459'],
 ['45892.09', '0.00322934'],
 ['45891.93', '0.43559330'],
 ['45889.02', '0.38381145'],
 ['45889.01', '0.19431292'],
 ['45889.00', '0.09057792'],
 ['45887.77', '0.43560722'],
 ['45887.74', '0.65300000'],
 ['45887.21', '0.65300000'],
 ['45886.13', '0.43400000'],
 ['45885.38', '0.10000000'],
 ['45881.94', '0.08772120'],
 ['45881.56', '1.09213924'],
 ['45881.55', '0.43562491'],
 ['45881.54', '0.01883678'],
 ['45880.12', '0.03880000'],
 ['45876.56', 

In [69]:
wsClient.all[2]['changes'][0][2]

'0.22400000'

In [43]:
wsClient

In [57]:
wsClient